In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import holodeck as holo
import holodeck.single_sources as ss
import holodeck.cyutils as cy
import matplotlib.pyplot as plt
import kalepy as kale


import matplotlib.cm as cm
from holodeck import plot, utils
from holodeck.constants import YR, MSOL, GYR

# Check flag replacement
## Fixed_Time

In [ ]:
dur, cad = 17.5*YR, 0.2*YR
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
sam = holo.sam.Semi_Analytic_Model()
# sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version
# hard = holo.hardening.Fixed_Time.from_sam(sam, 3*GYR)
# fobs_orb_edges = fobs_edges / 2.0 
# fobs_orb_cents = fobs/ 2.0
# edges, dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents) #, zero_stalled=True) # should the zero stalled option be part of the parameter space?
# edges[-1] = fobs_orb_edges


In [ ]:
# just using gwb() function
gwb = sam.gwb(fobs_edges, realize=10)
fig = plot.plot_gwb(fobs, gwb)

In [ ]:
# just using gwb() function

gwb = sam.gwb(fobs_edges, hard=holo.hardening.Hard_GW, realize=10)
fig = plot.plot_gwb(fobs, gwb)

Fixed_Time dadt issue: Need to create a Fixed_Time instance, like by hard = holo.hardening.Fixed_Time.from_sam(sam, 3*GYR)

# ss_gws_redz using sam.ss_gwb()
## with Hard_GW

In [ ]:
hc_ss, hc_bg, sspar, bgpar = sam.ss_gwb(fobs_edges, hard=holo.hardening.Hard_GW, realize = 10, loudest = 5, params = True)

In [ ]:
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)

In [ ]:
fig = plot.plot_pars(fobs, hc_ss, hc_bg, sspar, bgpar)

In [ ]:
hc_ss, hc_bg = sam.ss_gwb(fobs_edges, realize=30, loudest=100, params=False)
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)

## With Fixed_Time

In [ ]:
hard = holo.hardening.Fixed_Time.from_sam(sam, 3*GYR)
hc_ss, hc_bg, sspar, bgpar = sam.ss_gwb(fobs_edges, hard=hard, realize = 10, loudest = 5, params = True)
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)
fig = plot.plot_pars(fobs, hc_ss, hc_bg, sspar, bgpar )

In [ ]:
hc_ss, hc_bg = sam.ss_gwb(fobs_edges, hard=hard, realize = 10, loudest = 5, params = False)
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)

# Change redz for params

In [ ]:
dur, cad = 16*YR, 0.2*YR
fobs_gw_cents = utils.nyquist_freqs(dur,cad)
fobs_gw_edges = utils.nyquist_freqs_edges(dur,cad)
sam = holo.sam.Semi_Analytic_Model(mtot=(1e5*MSOL, 1e12*MSOL, 21), mrat = (0.001, 1, 16), redz=(0.001, 10, 11))
# sam=holo.sam.Semi_Analytic_Model()

hard = holo.hardening.Fixed_Time.from_sam(sam, 3*GYR)
# hard = holo.hardening.Hard_GW
# hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)

fobs_orb_cents = fobs_gw_cents / 2.0
fobs_orb_edges = fobs_gw_edges / 2.0
edges, dnum, redz_final = sam._dynamic_binary_number_at_fobs_consistent(hard, fobs_orb_cents)

In [ ]:
# edges, dnum, redz_final = sam._dynamic_binary_number_at_fobs_inconsistent(hard, fobs_orb_cents)
edges[-1] = fobs_orb_edges
print(redz_final.shape)

In [ ]:
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_gw_edges))

inside single_sources.ss_gws_redz()

In [ ]:
# hsfdf = hsamp^2 * f/df # this is same as hc^2
h2fdf = holo.gravwaves.char_strain_sq_from_bin_edges_redz(edges, redz_final)

In [ ]:
mt = kale.utils.midpoints(edges[0]) #: total mass
mr = kale.utils.midpoints(edges[1]) #: mass ratio
rz = kale.utils.midpoints(edges[2]) #: initial redshift

print('mt:', mt.shape, 'mr:', mr.shape, 'rz:', rz.shape)

# indices of bins sorted by h2fdf
indices = np.argsort(-h2fdf[...,0].flatten()) # just sort for first frequency
unraveled = np.array(np.unravel_index(indices, (len(mt),len(mr),len(rz))))
msort = unraveled[0,:]
qsort = unraveled[1,:]
zsort = unraveled[2,:]

In [ ]:
realize=10
loudest=5

hc2ss, hc2bg, lspar, bgpar, ssidx = \
                holo.cyutils.loudest_hc_and_par_from_sorted(number, h2fdf, realize, loudest,
                                                            mt, mr, rz, msort, qsort, zsort)

In [ ]:
print(ssidx.shape)
print([ssidx[0,...].shape, mt.shape])

In [ ]:
print(redz_final.shape)
redz = np.copy(redz_final)
for dd in range(3):
        redz = np.moveaxis(redz, dd, 0)
        redz = kale.utils.midpoints(redz, axis=0)
        redz = np.moveaxis(redz, 0, dd)
print(redz.shape)
print(mt.shape, mr.shape, rz.shape)

In [ ]:
# print(ssidx[0,...].shape)
# fidx = np.arange(len(ssidx[0]))
# print(fidx.shape)
# fidx = np.repeat(fidx, ssidx[0,0].size).reshape(ssidx[0].shape)
# print(fidx.shape)
# print(fidx[:,0,0])

# ssnew = np.array([])
# print(ssnew.shape)

In [ ]:
# ssnew = np.array([ssidx[0,...], ssidx[1,...], ssidx[2,...], ])

# par_rz_old = rz[ssidx[2,...]]
# par_rz_new = redz[ssidx[0,...], ssidx[1,...], ssidx[2,...]]

# print(par_rz_old.shape, par_rz_new.shape)

In [ ]:
qq = redz[ssidx[0,...], [ssidx[1,...], ssidx[2,...], ssidx[3,...]]]
print(qq.shape)
print(mt[ssidx[0,...]].shape)

In [ ]:
sspar = np.array([mt[ssidx[0,...]], mr[ssidx[1,...]], rz[ssidx[2,...]]])
for ff in range(len(ssidx[0])):
    for rr in range(len(ssidx[0,0])):
        for 

In [ ]:

# hc_ss, hc_bg, sspar, bgpar = sam.ss_gwb(fobs_gw_edges, hard, params=True)

In [ ]:
fobs_orb_edges = fobs_gw_edges / 2.0
fobs_orb_cents = fobs_gw_cents / 2.0

edges, dnum, redz_final = sam.dynamic_binary_number_at_fobs(hard, fobs_orb_cents)
edges[-1] = fobs_orb_edges

number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_gw_edges))
print(f"number: {utils.stats(number)}")
print(f"number.sum(): {number.sum():.4e}")

In [ ]:
use_redz = sam._redz_final